## Converting .txt files of patient atrium geometries to gmsh meshes .geo

Timothy Tyree

10.12.2019

In fipy, Gmsh2DIn3DSpace takes .geo files as arguments.  I have access to a (large registry of) patient atrium geometry in the form of tab delimated .txt files.  In order to simulate a model for atrial fibrillation on an atrium geometry using fipy, having a script to make those .geo files, then, is necessary.

It would be good to also take a given mesh and return the Delaunay triangulation of it, which could see slightly improved performance if the provided meshes have a lot of small angles, which is known to cause issues in fipy.

From wikipedia:
"In mathematics and computational geometry, a Delaunay triangulation (also known as a Delone triangulation) for a given set P of discrete points in a plane is a triangulation DT(P) such that no point in P is inside the circumcircle of any triangle in DT(P). Delaunay triangulations maximize the minimum angle of all the angles of the triangles in the triangulation; they tend to avoid sliver triangles. The triangulation is named after Boris Delaunay for his work on this topic from 1934."

pygmsh reccomends using the 'openCASCADE' kernel for more complicated meshes.  This will become necessary for higher spatial resolution atrium geometries.

In [1]:
import pygmsh as gm
import numpy as np

ModuleNotFoundError: No module named 'pygmsh'

In [2]:
#import example patient geometry .txts with np.loadtxt
patstr= 'pt85_RA'
vtdir = '../patients/'+patstr+'_vert'
fcdir = '../patients/'+patstr+'_faces'
vt = np.loadtxt(vtdir, delimiter=None, usecols=(0,1,2), dtype='float')
fc = np.loadtxt(fcdir, delimiter=None, usecols=(0,1,2), dtype='int')
fc = fc - 1 #fix the 1 indexing of input
#   TODO: make this interactive so I can manually reuse this easily
#   TODO: use 'openCASCADE' kernel for more complicated meshes
#   TODO: make this ipynb into a .py script that i can call over a database

NameError: name 'np' is not defined

In [88]:
#check that dimensions aggree
assert vt.shape[1] is fc.shape[1]
assert fc.flatten().max() <= vt.shape[0]
assert fc.flatten().min() >= 0
#calculate max pixel dimensions
dims = (np.max(vt[:,0]), np.max(vt[:,1]), np.max(vt[:,2]),
       np.min(vt[:,0]), np.min(vt[:,1]), np.min(vt[:,2]))
print('raw bounding cube: ' + str(dims))

raw bounding cube: (48.448502, -81.859642, 253.674362, 10.949102, -123.754646, 200.740952)


In [89]:
#initiate a gmsh object
geom = gm.built_in.Geometry()
#for each face in fc
for face in fc:
    #add the face to a gmsh object, assuming triangles are consistently oriented
    geom.add_polygon(vt[face-1], make_surface=True)

In [90]:
# generate and export .geo mesh
geodir= '../geo/'+patstr+'.geo'
mesh = gm.generate_mesh(geom, verbose=False, geo_filename=geodir)


geo file: ../geo/pt85_RA.geo


In [91]:
# once the mesh is create, it would be good to visualize it.
# TODO: make a file like this for .stl files regarding Eshom's project